In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 256

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)

trainset, validationset = random_split(dataset, [40000, 10000])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=2,pin_memory=True)
validationloader = torch.utils.data.DataLoader(validationset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=2,pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, prefetch_factor=2, num_workers=2,pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [56]:
def compute_accuracy(net, dataloader):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    
    for data in dataloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images.view(images.size(0), -1))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def numel(m: torch.nn.Module, only_trainable: bool = False):
    """
    returns the total number of parameters used by `m` (only counting
    shared parameters once); if `only_trainable` is True, then only
    includes parameters with `requires_grad = True`
    """
    parameters = m.parameters()
    if only_trainable:
        parameters = list(p for p in parameters if p.requires_grad)
    unique = dict((p.data_ptr(), p) for p in parameters).values()
    return sum(p.numel() for p in unique)

class LinearBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearBlock, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = nn.BatchNorm1d(output_dim)
    
    def forward(self, x):
        x = self.fc(x)
        x = F.relu(x)
        x = self.bn(x)
        
        return x

class ResLinearBlock(nn.Module):
    def __init__(self, input_dim, dim):
        super(ResLinearBlock, self).__init__()
        self.fc1 = LinearBlock(input_dim, dim)
        self.fc2 = LinearBlock(dim, input_dim)
        
    def forward(self, x):
        identity = x
        out = self.fc1(x)
        out = self.fc2(out)
        
        out += x    
        return out
    
class LinearNet(nn.Module):
    def __init__(self, input_dim ,output_dim, layers, block = LinearBlock, num_classes=10):
        super(LinearNet, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.layers = layers
        self.block = block
        self.feed = self.make_layers()
        

    def make_layers(self):
        network_layers = []
        network_layers.append(
            LinearBlock(self.input_dim, self.layers[0])
        )
        
        for i in range(len(self.layers) - 1):
            network_layers.append(
                self.block(self.layers[0], self.layers[i+1])
            )
            
        network_layers.append(
            nn.Linear(self.layers[0], self.output_dim)
        )
        

        return nn.Sequential(*network_layers)
    
    def forward(self, x):
        x = self.feed(x)
        return x
    


In [64]:
layers = [256 for i in range(8)] + [512 for i in range(8)] + [1024 for i in range(12)]
net = LinearNet(32*32*3, 10, layers, block = ResLinearBlock)

print(numel(net))
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
net = net.cuda(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())#, #momentum=0.9)

train_accuracy = []
validation_accuracy = []

validation_loss = []
train_loss = []

10171146


In [65]:
#Training Loop
for epoch in range(150):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device)

        # wrap them in Variable
        #inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.view(inputs.size(0), -1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    #if epoch % 10== 0 and epoch!=0 :
        #torch.save(net.state_dict(), f"./shallow_linear_3_regular_{epoch}")
        
        
    if epoch % 5 == 0:
        with torch.no_grad(): 
            v_loss = 0.0
            for j, data in enumerate(validationloader, 0):
                inputs, labels = data

                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outputs = net(inputs.view(inputs.size(0), -1))
                v_loss += criterion(outputs, labels).item()
                
            validation_loss.append(v_loss/j)
            
            v_accuracy = compute_accuracy(net, validationloader)
            validation_accuracy.append(v_accuracy)
            
            train_loss.append(running_loss/i)
            t_accuracy = compute_accuracy(net, trainloader)
            train_accuracy.append(t_accuracy)
    
    print(f"Epoch: {epoch}, training loss: {running_loss/i}, validation loss: {v_loss/j}, validation accuracy: {v_accuracy}%")
                
print('Finished Training')

Epoch: 0, training loss: 2.3392046888669333, validation loss: 1.988910008699466, validation accuracy: 34.99%
Epoch: 1, training loss: 1.767229370581798, validation loss: 1.988910008699466, validation accuracy: 34.99%
Epoch: 2, training loss: 1.5780096359741993, validation loss: 1.988910008699466, validation accuracy: 34.99%
Epoch: 3, training loss: 1.4751014923438048, validation loss: 1.988910008699466, validation accuracy: 34.99%
Epoch: 4, training loss: 1.4720748586532397, validation loss: 1.988910008699466, validation accuracy: 34.99%
Epoch: 5, training loss: 1.4020152963124788, validation loss: 1.5180623286809676, validation accuracy: 48.18%
Epoch: 6, training loss: 1.2851830140138283, validation loss: 1.5180623286809676, validation accuracy: 48.18%
Epoch: 7, training loss: 1.194488477630493, validation loss: 1.5180623286809676, validation accuracy: 48.18%
Epoch: 8, training loss: 1.1041421458507195, validation loss: 1.5180623286809676, validation accuracy: 48.18%
Epoch: 9, trainin

KeyboardInterrupt: 

In [74]:
import matplotlib.pyplot as plt

epochs_plot = [5*(i + 1) for i in range(epoch//5 + 1)]

plt.plot(epochs_plot[:-3], validation_loss[:-3])
plt.plot(epochs_plot[:-3], train_loss[:-3])
plt.title("Loss Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Validation Loss", "Training Loss"])
#plt.show()
plt.savefig("linear_deep_resnet_75_loss.png", dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

plt.clf()

plt.plot(epochs_plot[:-3], validation_accuracy[:-3])
plt.plot(epochs_plot[:-3], train_accuracy[:-2])
plt.title("Accuracy Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Validation Accuracy", "Training Accuracy"])
#plt.show()
plt.savefig("linear_deep_resnet_75_acc.png", dpi=None, facecolor='w', edgecolor='w',
        orientation='portrait', papertype=None, format=None,
        transparent=False, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)

plt.clf()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "frameon" which is no longer supported as of 3.3 and will become an error two minor releases later
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "papertype" which is no longer supported as of 3.3 and will become an error two minor releases later
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "frameon" which is no longer supported as of 3.3 and will become an e

<Figure size 432x288 with 0 Axes>

In [32]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        # calculate outputs by running images through the network
        outputs = net(images.view(images.size(0), -1))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 46 %


In [7]:
2%1


0

In [35]:
inputs.view(inputs.size(0), -1).shape

torch.Size([256, 3072])

In [72]:
epochs_plot[:-3]

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]